# Toronto Dwellings Analysis Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions to create a Panel dashboard.

In [ ]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [ ]:
# Initialize the Panel Extensions (for Plotly)
import panel as pn
pn.extension("plotly")

In [ ]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [ ]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/toronto_neighbourhoods_census_data.csv")
to_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/toronto_neighbourhoods_coordinates.csv")
df_neighbourhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

### Global available data

In [ ]:
# Getting the data from the top 10 expensive neighbourhoods
# YOUR CODE HERE!
def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    
    to_average_by_neighbourhood = to_data.groupby(by=["neighbourhood"]).mean()
    to_average_by_neighbourhood_descending = to_average_by_neighbourhood.sort_values('average_house_value', ascending=False)
    to_average_by_neighbourhood = to_average_by_neighbourhood_descending.drop(columns = ['single_detached_house', 'apartment_five_storeys_plus', 'movable_dwelling', 'semi_detached_house', 'row_house', 'duplex', 'apartment_five_storeys_less', 'other_house', 'shelter_costs_owned', 'shelter_costs_rented'])

    to_average_by_neighbourhood_top10 = to_average_by_neighbourhood_descending[:10].drop(columns = ['single_detached_house', 'apartment_five_storeys_plus', 'movable_dwelling', 'semi_detached_house', 'row_house', 'duplex', 'apartment_five_storeys_less', 'other_house', 'shelter_costs_owned', 'shelter_costs_rented'])
    plot = to_average_by_neighbourhood_top10.plot.bar()

    return plot

# Calculate the mean number of dwelling types units per year
# YOUR CODE HERE!
def housing_units_per_year():
    """Housing Units Per Year."""
    dwelling_data = [
    to_data.groupby('year').single_detached_house.sum(),
    to_data.groupby('year').apartment_five_storeys_plus.sum(),
    to_data.groupby('year').movable_dwelling.sum(),
    to_data.groupby('year').semi_detached_house.sum(),
    to_data.groupby('year').row_house.sum(),
    to_data.groupby('year').duplex.sum(),
    to_data.groupby('year').apartment_five_storeys_less.sum(),
    to_data.groupby('year').other_house.sum()

    ]

    dwelling_df = pd.DataFrame(data = dwelling_data)
    dwelling_df = dwelling_df.transpose()

    bar_chart = dwelling_df.groupby('year')

    plot = bar_chart.plot(kind = 'bar', figsize = (10,5))

    return plot


# Calculate the average monthly shelter costs for owned and rented dwellings
# YOUR CODE HERE!
def average_monthly_shelter_costs():
    """Average Monthly Shelter Costs"""
    shelter_data = [
    to_data.groupby('year').shelter_costs_owned.mean(),
    to_data.groupby('year').shelter_costs_rented.mean()
    ]

    shelter_data_df = pd.DataFrame(data = shelter_data)
    shelter_data_df = shelter_data_df.transpose()

    #plots 
    owned_dwellings = shelter_data_df['shelter_costs_owned']

    plot = owned_dwellings.plot.line(legend = True)

    return plot

    rented_dwellings = shelter_data_df['shelter_costs_rented']
    plot = rented_dwellings.plot.line(legend = True)
    
    return plot




### Panel Visualization Functions

In [ ]:
# Define Panel visualization functions
def neighbourhood_map():
    """Neighbourhood Map"""
   
    # YOUR CODE HERE!
    to_average_by_neighbourhood.reset_index(inplace = True)
    to_average_by_neighbourhood.set_index('neighbourhood', inplace = True)
    to_average_by_neighbourhood.head()
    
    map_data = pd.concat([df_neighbourhood_locations, to_average_by_neighbourhood], axis = 1, join = 'inner')
    map_data = map_data.drop(columns = 'index', axis=1)
    
    plot = px.scatter_mapbox(data_frame = map_data, lat = 'lat', lon = 'lon', color = 'average_house_value')

    return plot
    
    

def create_bar_chart(data, title, xlabel, ylabel, color):
    """
    Create a barplot based in the data argument.
    """
    
    # YOUR CODE HERE!
    bar_chart = dwelling_df.groupby('year')
    plot = bar_chart.plot(kind = 'bar', figsize = (10,5))
    
    return plot

def create_line_chart(data, title, xlabel, ylabel, color):
    """
    Create a line chart based in the data argument.
    """
    
    # YOUR CODE HERE!
    shelter_data = [
    to_data.groupby('year').shelter_costs_owned.mean(),
    to_data.groupby('year').shelter_costs_rented.mean()
    ]

    shelter_data_df = pd.DataFrame(data = shelter_data)
    shelter_data_df = shelter_data_df.transpose()
    
    owned_dwellings = shelter_data_df['shelter_costs_owned']
    plot1 = owned_dwellings.plot.line(legend = True)
    
    rented_dwellings = shelter_data_df['shelter_costs_rented']
    plot2 = rented_dwellings.plot.line(legend = True)
    
    return(plot1,plot2)

def average_house_value():
    """Average house values per year."""
    
    # YOUR CODE HERE!
    house_value = to_data.groupby('year').average_house_value.mean()
    house_value_df = pd.DataFrame(data = house_value)  
    
    plot1=house_value_df.plot(kind = 'line', figsize = (10,5))
    plot2=house_value_df.plot(kind = 'bar', figsize = (10,5))
    
    return(plot1,plot2)



def average_value_by_neighbourhood():
    """Average house values by neighbourhood."""
    
    # YOUR CODE HERE!
    neighbourhood_house_data = to_data.groupby(by=['year', 'neighbourhood']).mean()
    neighbourhood_house_df = pd.DataFrame(data = neighbourhood_house_data)
    neighbourhood_house_df = neighbourhood_house_df.drop(['single_detached_house', 'apartment_five_storeys_plus', 'movable_dwelling', 'semi_detached_house', 'row_house', 'duplex', 'apartment_five_storeys_less', 'other_house', 'shelter_costs_owned', 'shelter_costs_rented'], axis = 1)

    plot = neighbourhood_house_df.hvplot(kind='line', x = 'year', y = 'average_house_value', groupby='neighbourhood')


    return plot


def number_dwelling_types():
    """Number of dwelling types per year"""
    
    # YOUR CODE HERE!
    sum_data = to_data.groupby(by=['year', 'neighbourhood']).sum()
    plot_df = pd.DataFrame(data = sum_data)
    plot_df = plot_df.drop(['shelter_costs_owned', 'shelter_costs_rented', 'average_house_value'], axis = 1)

    plot = plot_df.hvplot(kind='bar', x = 'year', groupby='neighbourhood')
    
    return plot

def average_house_value_snapshot():
    """Average house value for all Toronto's neighbourhoods per year."""

    # YOUR CODE HERE!

def top_most_expensive_neighbourhoods():
    """Top 10 most expensive neighbourhoods."""
    
    # YOUR CODE HERE!
    to_average_by_neighbourhood = to_data.groupby(by=["neighbourhood"]).mean()
    to_average_by_neighbourhood_descending = to_average_by_neighbourhood.sort_values('average_house_value', ascending=False)
    to_average_by_neighbourhood = to_average_by_neighbourhood_descending.drop(columns = ['single_detached_house', 'apartment_five_storeys_plus', 'movable_dwelling', 'semi_detached_house', 'row_house', 'duplex', 'apartment_five_storeys_less', 'other_house', 'shelter_costs_owned', 'shelter_costs_rented'])

    to_average_by_neighbourhood_top10 = to_average_by_neighbourhood_descending[:10].drop(columns = ['single_detached_house', 'apartment_five_storeys_plus', 'movable_dwelling', 'semi_detached_house', 'row_house', 'duplex', 'apartment_five_storeys_less', 'other_house', 'shelter_costs_owned', 'shelter_costs_rented'])

    plot = to_average_by_neighbourhood_top10.plot.bar()
    
    return plot

def sunburts_cost_analysis():
    """Sunburst chart to conduct a costs analysis of most expensive neighbourhoods in Toronto per year."""
    
    # YOUR CODE HERE!

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [ ]:
# Create a Title for the Dashboard
# YOUR CODE HERE!


# Define a welcome text
# YOUR CODE HERE!

welcome = (f"Welcome to a visual representaiton of the Toronto Housing Data", neighbourhood_map)

# Create a tab layout for the dashboard
#Columns First 
# YOUR CODE HERE!
Dwelling_Types = pn.Column(
    "## Dwelling Type", create_bar_chart, number_dwelling_types)

Property_Values = pn.Column(
    "## Property Value Data", top_most_expensive_neighbourhoods, average_house_value, average_value_by_neighbourhood
)

Shelter_Data = pn.Column(
    "## Shelter Data", create_line_chart)

#Tabs and Dashboard Creation
Dashboard = pn.Tabs("Housing Data", Dwelling_Types, Property_Values, Shelter_Data)

# Create the main dashboard

# YOUR CODE HERE!

## Serve the Panel Dashboard

In [ ]:
Dashboard.servable()

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
# neighbourhood_map().show()

In [ ]:
# create_bar_chart(data, title, xlabel, ylabel, color)

# # Bar chart for 2001
# create_bar_chart(df_dwelling_units.loc[2001], "Dwelling Types in Toronto in 2001", "2001", "Dwelling Type Units", "red")

# # Bar chart for 2006
# create_bar_chart(df_dwelling_units.loc[2006], "Dwelling Types in Toronto in 2006", "2006", "Dwelling Type Units", "blue")

# # Bar chart for 2011
# create_bar_chart(df_dwelling_units.loc[2011], "Dwelling Types in Toronto in 2011", "2011", "Dwelling Type Units", "orange")

# # Bar chart for 2016
# create_bar_chart(df_dwelling_units.loc[2016], "Dwelling Types in Toronto in 2016", "2016", "Dwelling Type Units", "magenta")

In [ ]:
# create_line_chart(data, title, xlabel, ylabel, color)

# # Line chart for owned dwellings
# create_line_chart(df_avg_costs["shelter_costs_owned"], "Average Monthly Shelter Cost for Owned Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "blue")

# # Line chart for rented dwellings
# create_line_chart(df_avg_costs["shelter_costs_rented"], "Average Monthly Shelter Cost for Rented Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "orange")

In [ ]:
# average_house_value()

In [ ]:
# average_value_by_neighbourhood()

In [ ]:
# number_dwelling_types()

In [ ]:
# average_house_value_snapshot()

In [ ]:
# top_most_expensive_neighbourhoods()

In [ ]:
# sunburts_cost_analysis()